# Probability and Statistics
## Computers Lab 4
## Goodness of Fit Tests
------


# Report

In this lab we have learnt how to implement three different **Goodness of Fit** tests:

- Chi-Squared Goodness of Fit Test
- Chi-Squared Goodness of Fit for Independence
- Kolmogorov-Smirnov Test

The procedure for each test is the following:


#### Chi-Squared Goodness of Fit Test

1. From ```scipy.stats```, we can import the chi squared distribution (```chisquared```). Then, we have the function ```chisquare()``` available. With this function we can obtain test statistic and the p-value of the data
    
2. We group our observations into _bins_, and compare the resulting frequencies with the theoretical ones given by the _chi squared_ distribution. In order to group the data into _bins_, we can use ```numpy``` :
    

In [28]:
import numpy as np
from scipy.stats import chisquare
from sklearn import datasets #we import this to get some example data

data_set = datasets.load_breast_cancer()
y=data_set.data[:,0] #example data
size = len(y)

# Set up 50 bins for chi-square test
# Observed data will be approximately evenly distrubuted across all bins
percentile_bins = np.linspace(0,100,51) #creating the bins
percentile_cutoffs = np.percentile(y, percentile_bins) #separating the data (y) into bins
observed_frequency, bins = (np.histogram(y, bins=percentile_cutoffs))#get the frequency for each bin
cum_observed_frequency = np.cumsum(observed_frequency) #cumulative sum of frequency

3. Fit the data to a normal with the ```norm``` function from ```scipy.stats```:

In [29]:
params= stats.norm.fit(y) # the mean and standard deviation will be stored in params

4. Generate the expected frequencies. We can do this by getting the probability for each interval (with the cdf) and multiplying by the sample size. This can be done in python using a loop.

In [30]:
cdf_fitted = stats.norm.cdf(percentile_cutoffs,params[0], params[1])
expected_frequency = []
for bin in range(len(percentile_bins)-1):
    expected_cdf_area = cdf_fitted[bin+1] - cdf_fitted[bin]
    expected_frequency.append(expected_cdf_area)
expected_frequency = np.array(expected_frequency) * size

5. Perform the test using the ```chisquared()``` function, the observed frequency and the expected frequencies. See example below:

In [31]:
#chi-squared test
t_stat,p_value= chisquare(observed_frequency, f_exp=expected_frequency, ddof=2) #ddof=2, we have estimated two parameters -> mean and sd fo for normal dist.
print ('t-stat:', round(t_stat,3))
print ('p-value:', p_value)
#
df=len(observed_frequency)-1-len(params)
verif_p_val=1-stats.chi2.cdf(t_stat,df)
print ('verif p-val:',verif_p_val)
alpha=0.01
critical_value=stats.chi2.ppf(1-alpha,df)
Reject = (t_stat > stats.chi2.ppf(1-alpha,df))
print ('Reject',Reject)

t-stat: 151.068
p-value: 7.503154454169483e-13
verif p-val: 7.502887200416808e-13
Reject True


#### Chi-Squared Goodness of Fit for Independence

1. For this test we need to import ```chi2_contingency``` and ```chi2``` from ```scipy.stats```, and we also need ```pandas``` for dataframe management. 

2. Generate the contingency table as a ```pandas``` dataframe:


In [32]:
from scipy.stats import chi2_contingency
from scipy.stats import chi2
import pandas as pd
#Consider the contingency table:
tshirts = pd.DataFrame(
    [
        [48,22,33,47],
        [35,36,42,27]
    ],
    index=["Male","Female"],
    columns=["Black","White","Red","Blue"])
tshirts

,Black,White,Red,Blue
Male,48,22,33,47
Female,35,36,42,27


3. Perform the test using the ```chi2_contingency()``` function. This function returns _Q_, the _p-value_, the _degrees of freedom_ for the test and the _expected frequencies_.

In [33]:
Q, p_value, dof, ex = chi2_contingency(tshirts)
print('test_statistic=',Q)
print('p_value=',p_value)
print('degrees of freedom=',dof)

test_statistic= 11.56978992417547
p_value= 0.00901202511379703
degrees of freedom= 3


4. We can end the test here, or can also print the expected frequencies by including them in another dataframe:

In [34]:
print('expected frequencies:')
pd.DataFrame(
    data=ex[:,:], 
    index=["Male","Female"],
    columns=["Black","White","Red","Blue"]
).round(2)

expected frequencies:


,Black,White,Red,Blue
Male,42.93,30.0,38.79,38.28
Female,40.07,28.0,36.21,35.72


#### Kolmogorov-Smirnov Test

We can run this test simply with ```stats``` from ```scipy```. First, we generate the data and store it in a variable. Then, we can use the function ```stats.kstest()``` to test if the data fits to a normal distribution (or another of the available distributions). An example is shown below:

1. Generate the data:

In [35]:
from scipy import stats
x = np.linspace(-15, 15, 9)

2. Run the test:

In [36]:
D, p_value = stats.kstest(x, 'norm')
print('D statistic =',D)
print('p-value=',p_value)

D statistic = 0.4443560271592436
p-value= 0.03885014008678811


We can also test against other distributions by changing `'norm'` to the name of the distribution to be tested. Some distributions available in `scipy.stats` are: `'beta'`, `'expon'`, `'gamma'`, `'lognorm'`, `'norm'`, `'pearson3'`, `'triang'`, `'uniform'`, `'weibull_min'`, `'weibull_max'`.